# 临床试验 & 商业分析

In [1]:
import asyncio
import sys
from datetime import datetime
from dataclasses import asdict

sys.path.append('.')

from agent_core.agents.specialists.clinical_expert import ClinicalExpert
from agent_core.agents.specialists.commercial_expert import CommercialExpert
from agent_core.config.analysis_config import ConfigManager

In [2]:
async def analyze_clinical_commercial(genes, mode='standard', disease='metabolic disease'):
    """分析临床和商业数据 - 生成详细报告
    
    Args:
        genes: 基因列表
        mode: 'quick'/'standard'/'deep'
        disease: 疾病领域（商业分析需要）
    """
    # 获取配置
    if mode == 'quick':
        config = ConfigManager.get_quick_config()
    elif mode == 'standard':
        config = ConfigManager.get_standard_config()
    else:
        config = ConfigManager.get_deep_config()
    
    # 初始化专家
    clinical_expert = ClinicalExpert(config)
    commercial_expert = CommercialExpert(config)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    saved_files = []
    
    for i, gene in enumerate(genes, 1):
        print(f"\n[{i}/{len(genes)}] 分析 {gene}")
        
        filename = f"{gene}_detailed_analysis_{timestamp}.md"
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"# {gene} 靶点详细分析报告\n\n")
            f.write(f"生成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"分析模式: {mode}\n")
            f.write(f"数据来源: ClinicalTrials.gov + 商业数据库\n\n")
            
            # ======= 临床试验详细分析 =======
            try:
                print(f"  🏥 临床试验分析中...")
                result = await clinical_expert.analyze(gene)
                
                f.write(f"## 📊 临床试验分析\n\n")
                
                # 处理对象结果
                if hasattr(result, '__dict__'):
                    # 转换为字典
                    result_dict = asdict(result) if hasattr(result, '__dataclass_fields__') else result.__dict__
                    
                    # 基本统计
                    f.write(f"### 试验概况\n")
                    f.write(f"- **总试验数**: {result_dict.get('total_trials', 0)} 个\n")
                    f.write(f"- **活跃试验**: {result_dict.get('active_trials', 0)} 个\n")
                    f.write(f"- **已完成试验**: {result_dict.get('completed_trials', 0)} 个\n")
                    f.write(f"- **实际分析试验数**: {result_dict.get('analyzed_trials', 0)} 个\n")
                    f.write(f"- **置信度评分**: {result_dict.get('confidence_score', 0):.2f}\n\n")
                    
                    # 阶段分布
                    if 'phase_distribution' in result_dict and result_dict['phase_distribution']:
                        f.write(f"### 试验阶段分布\n")
                        phase_dist = result_dict['phase_distribution']
                        total = sum(phase_dist.values())
                        for phase, count in sorted(phase_dist.items()):
                            percentage = (count/total*100) if total > 0 else 0
                            f.write(f"- **{phase}**: {count} 个 ({percentage:.1f}%)\n")
                        f.write("\n")
                    
                    # 状态分布
                    if 'status_distribution' in result_dict and result_dict['status_distribution']:
                        f.write(f"### 试验状态分布\n")
                        for status, count in result_dict['status_distribution'].items():
                            f.write(f"- **{status}**: {count} 个\n")
                        f.write("\n")
                    
                    # 适应症分布
                    if 'condition_distribution' in result_dict and result_dict['condition_distribution']:
                        f.write(f"### 主要适应症\n")
                        conditions = list(result_dict['condition_distribution'].items())
                        conditions.sort(key=lambda x: x[1], reverse=True)
                        for condition, count in conditions[:10]:  # 显示前10个
                            f.write(f"- {condition}: {count} 个试验\n")
                        f.write("\n")
                    
                    # 地理分布
                    if 'geographic_distribution' in result_dict and result_dict['geographic_distribution']:
                        f.write(f"### 地理分布\n")
                        geo_dist = result_dict['geographic_distribution']
                        for region, data in geo_dist.items():
                            if isinstance(data, dict):
                                f.write(f"- **{region}**: {data.get('count', 0)} 个试验\n")
                            else:
                                f.write(f"- **{region}**: {data} 个试验\n")
                        f.write("\n")
                    
                    # 时间趋势
                    if 'temporal_trends' in result_dict and result_dict['temporal_trends']:
                        f.write(f"### 时间趋势分析\n")
                        trends = result_dict['temporal_trends']
                        if 'yearly_trial_starts' in trends:
                            f.write("**年度新增试验数**:\n")
                            for year, count in sorted(trends['yearly_trial_starts'].items())[-5:]:
                                f.write(f"- {year}: {count} 个\n")
                        if 'trend_analysis' in trends:
                            f.write(f"\n**趋势分析**: {trends['trend_analysis']}\n")
                        f.write("\n")
                    
                    # 关键试验
                    if 'key_trials' in result_dict and result_dict['key_trials']:
                        f.write(f"### 关键试验\n")
                        for trial in result_dict['key_trials'][:5]:  # 显示前5个
                            if isinstance(trial, dict):
                                f.write(f"#### {trial.get('title', 'N/A')}\n")
                                f.write(f"- **NCT号**: {trial.get('nct_id', 'N/A')}\n")
                                f.write(f"- **阶段**: {trial.get('phase', 'N/A')}\n")
                                f.write(f"- **状态**: {trial.get('status', 'N/A')}\n")
                                f.write(f"- **发起方**: {trial.get('sponsor', 'N/A')}\n")
                                if trial.get('condition'):
                                    f.write(f"- **适应症**: {trial.get('condition')}\n")
                                if trial.get('enrollment'):
                                    f.write(f"- **计划入组**: {trial.get('enrollment')} 人\n")
                                f.write("\n")
                    
                    # 分析总结
                    if 'summary' in result_dict and result_dict['summary']:
                        f.write(f"### 临床开发总结\n")
                        f.write(result_dict['summary'] + "\n\n")
                    
                    print(f"     ✓ 详细分析 {result_dict.get('total_trials', 0)} 个试验")
                    
                else:
                    # 字典格式
                    f.write(f"- 总试验数: {result.get('total_trials', 0)}\n\n")
                    if 'report' in result:
                        f.write(result['report'] + "\n\n")
                    
            except Exception as e:
                f.write(f"## 临床试验分析\n❌ 错误: {str(e)}\n\n")
                print(f"     ✗ 失败: {e}")
            
            # ======= 商业分析 =======
            try:
                print(f"  💰 商业分析中...")
                result = await commercial_expert.analyze(gene, disease)
                
                f.write(f"## 💼 商业化分析\n\n")
                f.write(f"**疾病领域**: {disease}\n\n")
                
                # 处理返回结果
                if hasattr(result, '__dict__'):
                    result_dict = asdict(result) if hasattr(result, '__dataclass_fields__') else result.__dict__
                    
                    # 市场规模
                    if 'market_size' in result_dict:
                        f.write(f"### 市场规模\n")
                        f.write(f"{result_dict['market_size']}\n\n")
                    
                    # 竞争格局
                    if 'competitive_landscape' in result_dict:
                        f.write(f"### 竞争格局\n")
                        f.write(f"{result_dict['competitive_landscape']}\n\n")
                    
                    # 商业价值
                    if 'commercial_value' in result_dict:
                        f.write(f"### 商业价值评估\n")
                        f.write(f"{result_dict['commercial_value']}\n\n")
                    
                    # 完整报告
                    if 'report' in result_dict:
                        f.write(f"### 详细分析\n")
                        f.write(result_dict['report'] + "\n\n")
                        
                elif isinstance(result, str):
                    f.write(result + "\n\n")
                else:
                    f.write(str(result) + "\n\n")
                    
                print(f"     ✓ 商业分析完成")
                
            except Exception as e:
                f.write(f"## 商业化分析\n❌ 错误: {str(e)}\n\n")
                print(f"     ✗ 失败: {e}")
            
            # ======= 综合建议 =======
            f.write(f"## 🎯 综合建议\n\n")
            f.write(f"基于 {gene} 的临床试验现状和商业潜力分析:\n\n")
            f.write(f"1. **临床开发策略**: 根据现有试验阶段分布，建议关注未覆盖的适应症\n")
            f.write(f"2. **市场定位**: 结合疾病领域 ({disease}) 的市场需求进行差异化定位\n")
            f.write(f"3. **合作机会**: 寻找活跃试验的发起方进行潜在合作\n")
            f.write(f"4. **风险评估**: 关注已失败或终止的试验，了解潜在风险因素\n\n")
        
        saved_files.append(filename)
        print(f"  📄 详细报告已保存: {filename}")
    
    print(f"\n✅ 完成! 共生成 {len(saved_files)} 份详细报告:")
    for f in saved_files:
        print(f"  • {f}")
    
    return saved_files

In [ ]:
# 使用示例
my_genes = ['PNPLA3', 'MTARC1', 'DGAT2', 'FAM13A']

# 运行分析
saved_files = await analyze_clinical_commercial(
    genes=my_genes,
    mode='standard',  # 'quick'/'standard'/'deep'
    disease='NASH'  # 商业分析需要疾病信息，如：NASH, diabetes, obesity等
)

In [7]:
# 如果需要不同疾病的分析，可以自定义
gene_disease_pairs = [
    ('PNPLA3', '肥胖'),
    ('MTARC1', '肥胖'),
    ('DGAT2', ' 肥胖'),
    ('FAM13A', '肥胖')
]

for gene, disease in gene_disease_pairs:
    print(f"\n分析 {gene} - {disease}")
    await analyze_clinical_commercial(
        genes=[gene],
        mode='deep',
        disease=disease
    )


分析 PNPLA3 - 肥胖

[1/1] 分析 PNPLA3
  🏥 临床试验分析中...
     ✓ 详细分析 49 个试验
  💰 商业分析中...
     ✓ 商业分析完成
  📄 详细报告已保存: PNPLA3_detailed_analysis_20250828_175727.md

✅ 完成! 共生成 1 份详细报告:
  • PNPLA3_detailed_analysis_20250828_175727.md

分析 MTARC1 - 肥胖

[1/1] 分析 MTARC1
  🏥 临床试验分析中...


第1页请求失败: Cannot connect to host beta.clinicaltrials.gov:443 ssl:default [None]


     ✓ 详细分析 0 个试验
  💰 商业分析中...
     ✓ 商业分析完成
  📄 详细报告已保存: MTARC1_detailed_analysis_20250828_190728.md

✅ 完成! 共生成 1 份详细报告:
  • MTARC1_detailed_analysis_20250828_190728.md

分析 DGAT2 -  肥胖

[1/1] 分析 DGAT2
  🏥 临床试验分析中...
     ✓ 详细分析 5 个试验
  💰 商业分析中...


生成分析报告失败: Connection error.


     ✗ 失败: Connection error.
  📄 详细报告已保存: DGAT2_detailed_analysis_20250828_191053.md

✅ 完成! 共生成 1 份详细报告:
  • DGAT2_detailed_analysis_20250828_191053.md

分析 FAM13A - 肥胖

[1/1] 分析 FAM13A
  🏥 临床试验分析中...
     ✓ 详细分析 0 个试验
  💰 商业分析中...
     ✓ 商业分析完成
  📄 详细报告已保存: FAM13A_detailed_analysis_20250828_191512.md

✅ 完成! 共生成 1 份详细报告:
  • FAM13A_detailed_analysis_20250828_191512.md
